In [1]:
%matplotlib inline

import sys

sys.path.append('Scripts')
from datahandling import wofs_fuser

import datacube
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from pathlib import Path
from datacube.utils import masking
from datacube.utils import geometry 
from datacube.utils.geometry import CRS
from datacube.helpers import write_geotiff
import geopandas as gpd
import ipyleaflet as ipyl
import ipywidgets as ipyw
import json



dc = datacube.Datacube()

In [3]:
gdf = gpd.read_file('Reservoirs/Reservoirs.shp')

#gdf = gdf.to_crs({'init':'epsg:4326'}) #4326 is the tag for lat lon format
#we need to get the lat and lon of where the dam is to call the right satellite data later
#we want the lat and lon to be columns in the data to make it easy when we query the wofs data later
#Getting the centroid from the polygon and adding a latitude and longitude column (I found this code on the internet):
def getXY(pt):
    return (pt.x, pt.y)
centroidseries = gdf['geometry'].centroid
x,y = [list(t) for t in zip(*map(getXY, centroidseries))]

gdf = gdf.assign(lon=x)
gdf = gdf.assign(lat=y)
#gdf['info'] = list(zip(gdf.lat, gdf.lon, gdf.Area_calc))
gdf = gdf[['NAME', 'SHAPE_Leng', 'SHAPE_Area', 'Area_calc', 'lat', 'lon', 'geometry']]
gdf = gdf.dropna()
gdf

,NAME,SHAPE_Leng,SHAPE_Area,Area_calc,lat,lon,geometry
0,WILSON RESERVOIR,0.034915,0.000028,270561.736,-37.510018,144.018549,"POLYGON ((144.01821 -37.51455, 144.01719 -37.5..."
1,SUGARLOAF RESERVOIR,0.212903,0.000438,4284887.270,-37.672015,145.305467,"POLYGON ((145.31084 -37.68275, 145.31059 -37.6..."
2,GREEN LAKE,0.155045,0.000715,7111145.660,-36.435338,144.834187,"POLYGON ((144.82094 -36.43662, 144.82081 -36.4..."
3,LAKE TAYLOR,0.244253,0.000490,4853573.620,-36.783437,142.383614,"POLYGON ((142.39295 -36.77087, 142.39255 -36.7..."
4,UPPER STONY CREEK RESERVOIR,0.059762,0.000051,500287.353,-37.818353,144.203635,"POLYGON ((144.21140 -37.81367, 144.21152 -37.8..."
...,...,...,...,...,...,...,...
1017,KIRAR WEIR,0.257613,0.000240,2681688.020,-25.299279,151.063109,"POLYGON ((151.07836 -25.34552, 151.07834 -25.3..."
1018,MARIAN WEIR,0.266732,0.000134,1544632.180,-21.146641,148.885758,"POLYGON ((148.82987 -21.17723, 148.82926 -21.1..."
1019,MARY BARRAGE,0.192548,0.000115,1283147.610,-25.627152,152.613932,"POLYGON ((152.59687 -25.65457, 152.59709 -25.6..."
1020,TINANA BARRAGE,0.034002,0.000012,129235.966,-25.571684,152.713346,"POLYGON ((152.71974 -25.56929, 152.71973 -25.5..."


In [ ]:
#Converting the data to json
data = json.loads(gdf.to_json())

map = ipyl.Map(center=[-28, 148], zoom=7)

label = ipyw.Label(layout=ipyw.Layout(width='100%'))

for feature in data['features']:
    feature['properties']['style'] = {
        'color': 'grey',
        'weight': 1,
        'fillColor': 'grey',
        'fillOpacity': 0.5
    }
layer = ipyl.GeoJSON(data=data, hover_style={'fillColor': 'red'})

def click_handler(event=None, feature=None, id=None, properties=None):
    label.value = str(properties['NAME'])
    
    
layer.on_hover(click_handler)
map.add_layer(layer)


ipyw.VBox([map, label])

In [ ]:
label.value

In [4]:
output = 'Named_Reservoirs'
gdf.to_file(output)